In [69]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LassoCV
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ShuffleSplit
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import statsmodels.formula.api as smf
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.tree import DecisionTreeRegressor
from QBUS2820 import rmse_jack, r2_jack 
from sklearn.linear_model import Ridge 
from sklearn.linear_model import Lasso
from sklearn.linear_model import RidgeCV
import xgboost as xgb
from sklearn.grid_search import GridSearchCV
from ExtraCode import getResultTable
from sklearn.model_selection import cross_val_score
from QBUS2820 import forward
from sklearn.linear_model import ElasticNet
from QBUS2820 import pcrCV
from QBUS2820 import plsCV
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor

In [56]:
data = pd.read_csv('Train6_1.csv')
final_train = data.sample(frac=0.6, random_state=1)
final_test = data[data.index.isin(final_train.index)==False]
final_train.head()
y_train = final_train.pop('SalePrice')
y_test = final_test.pop('SalePrice')
y_train = np.log(y_train)



In [ ]:
pred = []
method = []

In [59]:
pcr=pcrCV(final_train, y_train)
predpcr = pcr.predict(final_test)
method.append('PCR')
pred.append(np.exp(predpcr))


In [71]:
knn1 = KNeighborsRegressor(n_neighbors=10) 
knn1.fit(final_train, y_train)

predknn = np.exp(knn1.predict(final_test))
method.append('KNN with 10')
pred.append(predknn)

### Testing Models

In [3]:
#Forward selection
fwd = forward()
fwd.fit(final_train, y_train)
predforward = fwd.predict(final_test)
method.append('Forward')
pred.append(np.exp(predforward))

GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)
GBoost.fit(final_train,y_train)
predFinalBoost = GBoost.predict(final_test)
method.append('GBoost')
pred.append(np.exp(predFinalBoost))

regr = AdaBoostRegressor(loss='linear', learning_rate = 1, n_estimators = 350)
regr = regr.fit(final_train,y_train)
adapred = regr.predict(final_test)
method.append('AdaBoost')
pred.append(np.exp(adapred))

In [4]:
xgdmat = xgb.DMatrix(final_train, y_train) # Create our DMatrix to make XGBoost more efficient
our_params = {'eta': 0.1, 'seed':0, 'subsample': 0.8, 'colsample_bytree': 0.8, 
             'objective': 'reg:linear', 'max_depth':3, 'min_child_weight':1} 

final_gb = xgb.train(our_params, xgdmat, num_boost_round = 432)
testFinaldmat = xgb.DMatrix(final_test)
xgpred = final_gb.predict(testFinaldmat)
method.append('XGBoost')
pred.append(np.exp(xgpred))

In [5]:
pred2 = (np.exp(xgpred)+np.exp(adapred)+np.exp(predFinalBoost))/3
pred.append(pred2)
method.append('XGBoost, ada, GB')

In [6]:
pred3 = (np.exp(xgpred)+np.exp(predFinalBoost))/2

pred5 = (np.exp(xgpred)+np.exp(adapred))/2
pred.append(pred3)
pred.append(pred5)
method.append('xg GB')
method.append('xg adaboost')

In [7]:
#Extremely Random forest
regr2 = ExtraTreesRegressor(criterion='mae',max_depth=None,min_samples_split=2)
regr2 = regr.fit(final_train,y_train)

predFinalExtRandomForestlad = regr2.predict(final_test)
pred.append(np.exp(predFinalExtRandomForestlad))
method.append('Random Forest')

In [8]:
pred5 = (np.exp(xgpred)+np.exp(predFinalExtRandomForestlad)+np.exp(predFinalBoost))/3
pred.append(pred5)
method.append('XGBoost, random tree, GB')

In [9]:
pred6 = (np.exp(xgpred)+np.exp(predFinalExtRandomForestlad))/2
pred.append(pred6)
method.append('xg Random tree')

In [10]:
pred10 = (np.exp(xgpred)+np.exp(predFinalBoost)+np.exp(predFinalExtRandomForestlad))/3
pred.append(pred10)
method.append('XGBoost, GBoost Random Forest')

In [11]:
#LASSO
lasso = Lasso(alpha = 1)
lasso.fit(final_train, np.ravel(y_train)) 
pred_L = lasso.predict(final_test)
method.append('LASSO')
pred.append(np.exp(pred_L))

In [12]:
alpha = 10
rg = Ridge(alpha = alpha)
rg.fit(final_train, y_train)
y_pred_rg = (rg.predict(final_test))
y_pred_rg

pred.append(np.exp(y_pred_rg))
method.append('Ridge alpha 10')

In [13]:
pred11 = (np.exp(xgpred)+np.exp(predFinalBoost)+np.exp(predFinalExtRandomForestlad)+np.exp(y_pred_rg))/4
pred.append(pred11)
method.append('XGBoost, GBoost Random Forest Ridge')

In [14]:
pred12 = (np.exp(xgpred)+np.exp(predFinalBoost)+np.exp(predFinalExtRandomForestlad)+np.exp(pred_L))/4
pred.append(pred12)
method.append('XGBoost, GBoost Random Forest LASSO')

In [15]:
regr = ElasticNet(random_state=0, alpha = 1)
    
regr.fit(final_train, y_train)    
pred13 = regr.predict(final_test)
pred.append(np.exp(pred13))
method.append('ENET')

In [16]:
pred12 = (np.exp(xgpred)+np.exp(predFinalBoost)+np.exp(predFinalExtRandomForestlad)+np.exp(pred13))/4
pred.append(pred12)
method.append('XGBoost, GBoost Random Forest ENET')

In [17]:
pred13 = (np.exp(xgpred)+np.exp(predFinalBoost)+np.exp(y_pred_rg))/3
pred.append(pred13)
method.append('XGBoost, GBoost Ridge')

In [18]:
pred14 = (np.exp(xgpred)+np.exp(predFinalBoost)+np.exp(y_pred_rg)+np.exp(predforward))/4
pred.append(pred14)
method.append('XGBoost, GBoost Ridge Forward')

In [19]:
pred15 = (np.exp(xgpred)+np.exp(predFinalBoost)+np.exp(y_pred_rg)+np.exp(predforward)+ np.exp(predFinalExtRandomForestlad))/5
pred.append(pred15)
method.append('XGBoost, GBoost Ridge Forward Tree')

In [20]:
pred16 = (np.exp(xgpred)+np.exp(predFinalBoost)+np.exp(y_pred_rg)+np.exp(predforward)+ np.exp(pred_L))/5
pred.append(pred16)
method.append('XGBoost, GBoost Ridge Forward Lasso')

In [21]:
pred15 = (np.exp(xgpred)+np.exp(predFinalBoost)+ np.exp(predFinalExtRandomForestlad))/3
pred.append(pred15)
method.append('XGBoost, GBoost, RandomTree')

In [22]:

pred15 = ((0.3*np.exp(xgpred))+(0.3*np.exp(predFinalBoost))+(0.2*np.exp(y_pred_rg))+(0.2*np.exp(predforward)))
pred.append(pred15)
method.append('XGBoost, GBoost Ridge Forward 30 30 20 20')

In [23]:
pred16 = ((0.4*np.exp(xgpred))+(0.4*np.exp(predFinalBoost))+(0.1*np.exp(y_pred_rg))+(0.1*np.exp(predforward)))
pred.append(pred16)
method.append('XGBoost, GBoost Ridge Forward 40 40 10 10')

In [24]:
pred17 = ((0.2*np.exp(xgpred))+(0.2*np.exp(predFinalBoost))+(0.3*np.exp(y_pred_rg))+(0.3*np.exp(predforward)))
pred.append(pred17)
method.append('XGBoost, GBoost Ridge Forward 20 20 30 30')

In [25]:
pred18 = ((0.35*np.exp(xgpred))+(0.35*np.exp(predFinalBoost))+(0.15*np.exp(y_pred_rg))+(0.15*np.exp(predforward)))
pred.append(pred18)
method.append('XGBoost, GBoost Ridge Forward 35 35 15 15')

In [26]:
pred19 = ((0.25*np.exp(xgpred))+(0.35*np.exp(predFinalBoost))+(0.2*np.exp(y_pred_rg))+(0.2*np.exp(predforward)))
pred.append(pred19)
method.append('XGBoost, GBoost Ridge Forward 25 35 20 20')

In [27]:
pred20 = ((0.2*np.exp(xgpred))+(0.4*np.exp(predFinalBoost))+(0.2*np.exp(y_pred_rg))+(0.2*np.exp(predforward)))
pred.append(pred20)
method.append('XGBoost, GBoost Ridge Forward 20 40 20 20')

In [28]:
pred21 = ((0.2*np.exp(xgpred))+(0.3*np.exp(predFinalBoost))+(0.2*np.exp(y_pred_rg))+(0.3*np.exp(predforward)))
pred.append(pred21)
method.append('XGBoost, GBoost Ridge Forward 20 30 20 30')

In [29]:
pred22 = ((0.2*np.exp(xgpred))+(0.3*np.exp(predFinalBoost))+(0.3*np.exp(y_pred_rg))+(0.2*np.exp(predforward)))
pred.append(pred22)
method.append('XGBoost, GBoost Ridge Forward 20 30 30 20')

In [30]:
pred23 = ((0.2*np.exp(xgpred))+(0.2*np.exp(predFinalBoost))+(0.3*np.exp(y_pred_rg))+(0.3*np.exp(predforward)))
pred.append(pred23)
method.append('XGBoost, GBoost Ridge Forward 20 20 30 30')

In [31]:
pred24 = ((0.3*np.exp(xgpred))+(0.5*np.exp(predFinalBoost))+(0.1*np.exp(y_pred_rg))+(0.1*np.exp(predforward)))
pred.append(pred24)
method.append('XGBoost, GBoost Ridge Forward 30 50 10 10')

In [32]:
pred24 = ((0.2*np.exp(xgpred))+(0.2*np.exp(predFinalBoost))+(0.2*np.exp(y_pred_rg))+(0.4*np.exp(predforward)))
pred.append(pred24)
method.append('XGBoost, GBoost Ridge Forward 20 20 20 40')

In [33]:
pred25 = ((0.3*np.exp(xgpred))+(0.3*np.exp(predFinalBoost))+(0.3*np.exp(y_pred_rg))+(0.1*np.exp(predforward)))
pred.append(pred25)
method.append('XGBoost, GBoost Ridge Forward 30 30 30 10')

In [34]:
pred26 = ((0.26*np.exp(xgpred))+(0.27*np.exp(predFinalBoost))+(0.27*np.exp(y_pred_rg))+(0.2*np.exp(predforward)))
pred.append(pred26)
method.append('XGBoost, GBoost Ridge Forward 26 27 27 20')

In [35]:
pred27 = ((0.23*np.exp(xgpred))+(0.24*np.exp(predFinalBoost))+(0.24*np.exp(y_pred_rg))+(0.3*np.exp(predforward)))
pred.append(pred27)
method.append('XGBoost, GBoost Ridge Forward 23 24 24 30')

In [74]:
getResultTable(method,pred)

,Test RMSE,SE,Jack R2,SE,MAE,R-square
Forward,23834.417,4933.468,0.884,0.047,14042.658,0.884
GBoost,19401.538,2816.862,0.923,0.015,12968.305,0.923
AdaBoost,26319.667,3692.210,0.858,0.025,18198.414,0.858
XGBoost,20400.397,3406.287,0.915,0.020,13619.232,0.915
"XGBoost, ada, GB",20452.570,3570.834,0.915,0.020,13540.463,0.915
xg GB,19250.267,3211.549,0.924,0.017,12745.760,0.924
xg adaboost,21852.740,3803.248,0.902,0.023,14624.290,0.902
Random Forest,25869.901,3510.182,0.863,0.023,17952.529,0.863
"XGBoost, random tree, GB",20264.068,3505.125,0.916,0.020,13440.667,0.916
xg Random tree,21582.591,3705.208,0.905,0.022,14419.793,0.905


In [73]:
def getResultTable(rows, predictions):
    columns=['Test RMSE', 'SE', 'Jack R2', 'SE', 'MAE', 'R-square']
    results=pd.DataFrame(0.0, columns=columns, index=rows)

    for row,pred in zip(range(0,len(rows)),predictions):
        results.iloc[row,0], results.iloc[row,1] = rmse_jack(y_test, pred)
        results.iloc[row,2], results.iloc[row,3] = (r2_jack(y_test, pred))
        results.iloc[row,4] = mean_absolute_error(y_test, pred)
        results.iloc[row,5] = r2_score(y_test,pred)
    return results.round(3)

## Kaggle Predictions

In [77]:
data = pd.read_csv('Train6_1.csv')
y_price = data.pop('SalePrice')
kaggle = pd.read_csv('Test6_1.csv')
y_price = np.log(y_price)
xgdmat = xgb.DMatrix(data, y_price) # Create our DMatrix to make XGBoost more efficient
testFinaldmat = xgb.DMatrix(kaggle)

In [78]:
our_params = {'eta': 0.1, 'seed':0, 'subsample': 0.8, 'colsample_bytree': 0.8, 
             'objective': 'reg:linear', 'max_depth':3, 'min_child_weight':1} 

final_gb = xgb.train(our_params, xgdmat, num_boost_round = 432)
prediction = final_gb.predict(testFinaldmat)

In [79]:
GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)
GBoost.fit(data,y_price)
predFinalBoost = GBoost.predict(kaggle)


In [80]:
alpha = 10
rg = Ridge(alpha = alpha)
rg.fit(data, y_price)
y_pred_rg = (rg.predict(kaggle))

In [81]:
#Forward selection
fwd = forward()
fwd.fit(data, y_price)
predforward = fwd.predict(kaggle)

In [47]:
finalprediction = ((0.2*np.exp(prediction))+(0.35*np.exp(predFinalBoost))+(0.2*np.exp(y_pred_rg))+(0.25*np.exp(predforward)))

In [48]:
#This is for the indices
ind = np.arange(1,1609)
headers = ['Id','Prediction']
prediction2 = pd.DataFrame({'Id':ind, 'Prediction':finalprediction})
#Saving results into CSV file 
prediction2.to_csv("Day10_1.csv", index=False)

In [50]:
finalprediction = ((0.15*np.exp(prediction))+(0.35*np.exp(predFinalBoost))+(0.2*np.exp(y_pred_rg))+(0.3*np.exp(predforward)))

In [51]:
#This is for the indices
ind = np.arange(1,1609)
headers = ['Id','Prediction']
prediction2 = pd.DataFrame({'Id':ind, 'Prediction':finalprediction})
#Saving results into CSV file 
prediction2.to_csv("Day10_2.csv", index=False)

In [52]:
finalprediction = ((0.1*np.exp(prediction))+(0.35*np.exp(predFinalBoost))+(0.25*np.exp(y_pred_rg))+(0.3*np.exp(predforward)))

In [53]:
#This is for the indices
ind = np.arange(1,1609)
headers = ['Id','Prediction']
prediction2 = pd.DataFrame({'Id':ind, 'Prediction':finalprediction})
#Saving results into CSV file 
prediction2.to_csv("Day10_3.csv", index=False)

In [54]:
finalprediction = ((0.15*np.exp(prediction))+(0.35*np.exp(predFinalBoost))+(0.15*np.exp(y_pred_rg))+(0.35*np.exp(predforward)))

In [55]:
#This is for the indices
ind = np.arange(1,1609)
headers = ['Id','Prediction']
prediction2 = pd.DataFrame({'Id':ind, 'Prediction':finalprediction})
#Saving results into CSV file 
prediction2.to_csv("Day10_4.csv", index=False)

In [82]:
#Extremely Random forest
regr2 = ExtraTreesRegressor(criterion='mae',max_depth=None,min_samples_split=2)
regr2 = regr.fit(data,y_price)

predFinalExtRandomForestlad = regr2.predict(kaggle)

In [83]:
finalprediction = ((0.15*np.exp(prediction))+(0.35*np.exp(predFinalBoost))+(0.1*np.exp(y_pred_rg))+ (predFinalExtRandomForestlad*0.05) + (0.35*np.exp(predforward)))

In [84]:
#This is for the indices
ind = np.arange(1,1609)
headers = ['Id','Prediction']
prediction2 = pd.DataFrame({'Id':ind, 'Prediction':finalprediction})
#Saving results into CSV file 
prediction2.to_csv("Day10_5.csv", index=False)

In [85]:
prediction2

,Id,Prediction
0,1,92072.823471
1,2,153179.049732
2,3,129355.663851
3,4,178094.097731
4,5,140120.653509
5,6,353037.400209
6,7,126564.163675
7,8,263930.578905
8,9,111575.424262
9,10,120892.795160
